# **Praktikum 10_2 - Recurrent Neural Network (RNN)**

Nama : Alfan Olivan

NIM : 2141720078

## **Praktikum 2**

**Setup**

In [1]:
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


**Load data**

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


**Olah teks**

  * Vectorize Teks

In [6]:
example_texts = ['abcdefg', 'xyz']
chars=tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [7]:
ids_from_chars = tf.keras.layers.StringLookup(
vocabulary=list(vocab),mask_token=None)

In [8]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [9]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [10]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [11]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [12]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

* Prediksi

In [13]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [14]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [15]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [16]:
seq_length=100

In [17]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [18]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [19]:
def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text

In [20]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [21]:
dataset=sequences.map(split_input_target)

In [22]:
for input_example, target_example in dataset.take(1):
    print("Input :",text_from_ids(input_example).numpy())
    print("Target:",text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


* Membuat Batch Training

In [23]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

**Membuat Model**

In [24]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [25]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [26]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

**Uji Model**

In [27]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [28]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
sampled_indices=tf.random.categorical(example_batch_predictions[0],num_samples=1)
sampled_indices=tf.squeeze(sampled_indices,axis=-1).numpy()

In [30]:
sampled_indices

array([22,  1, 50,  1, 44,  0,  0, 58, 49, 24, 35, 53, 10,  2, 41, 23, 27,
        7, 13, 14, 19, 46, 39,  3,  2, 13, 63, 10, 62, 25, 12, 13, 54,  3,
        5, 23, 28,  9, 57, 41, 15, 22, 43, 51, 65, 61, 50, 59, 62, 47, 64,
       17, 44, 57, 53, 48, 43, 53, 52, 26, 43, 18,  6, 32, 56, 21, 52, 48,
       58, 58, 12, 55, 31, 23, 38, 27, 60, 18, 51, 25, 62, 39,  7, 32, 17,
       36, 30, 52, 26, 60, 30,  2, 22, 56, 37, 18, 57, 56, 17, 24])

In [31]:
print("Input:\n",text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n",text_from_ids(sampled_indices).numpy())

Input:
 b" the gods had nothing else to do\nBut to confirm my curses! Could I meet 'em\nBut once a-day, it would"

Next Char Predictions:
 b"I\nk\ne[UNK][UNK]sjKVn3 bJN,?AFgZ! ?x3wL;?o!&JO.rbBIdlzvktwhyDernidnmMdE'SqHmiss;pRJYNuElLwZ,SDWQmMuQ IqXErqDK"


**Train Model**

In [32]:
loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [33]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1899014, shape=(), dtype=float32)


In [34]:
tf.exp(example_batch_mean_loss).numpy()

66.01628

In [35]:
model.compile(optimizer='adam',loss=loss)

In [36]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [37]:
EPOCHS=20
history=model.fit(dataset,epochs=EPOCHS,callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 14s 53ms/step - loss: 2.7165
Epoch 2/20
172/172 [==============================] - 11s 52ms/step - loss: 1.9819
Epoch 3/20
172/172 [==============================] - 13s 58ms/step - loss: 1.7043
Epoch 4/20
172/172 [==============================] - 14s 55ms/step - loss: 1.5436
Epoch 5/20
172/172 [==============================] - 11s 53ms/step - loss: 1.4457
Epoch 6/20
172/172 [==============================] - 11s 55ms/step - loss: 1.3778
Epoch 7/20
172/172 [==============================] - 11s 54ms/step - loss: 1.3266
Epoch 8/20
172/172 [==============================] - 12s 55ms/step - loss: 1.2819
Epoch 9/20
172/172 [==============================] - 14s 58ms/step - loss: 1.2403
Epoch 10/20
172/172 [==============================] - 11s 55ms/step - loss: 1.2005
Epoch 11/20
172/172 [==============================] - 11s 55ms/step - loss: 1.1609
Epoch 12/20
172/172 [==============================] - 11s 56ms/step - loss: 1.1195
E

**Generate Teks**

In [38]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [39]:
one_step_model=OneStep(model, chars_from_ids, ids_from_chars)

In [40]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
If he holds here at thy father's brows
Affection, for my pardon, I will plot emir morn.
Your thief thinkless, uncle may; served it so man
And a desire of it our Bohemia

FLORIZEL:
O sea!
Why most you been proved a friend of little.

PAULINA:
I do beseech you.

CORIOLANUS:
No more.

First Senator:
No, if I know 'tis man's his brother's life!

All:
An if he marr'd
The open air, which plead'st thou owest no wife.

BAPTISTA:
How aged as toucheth use to fear, banisher,
Our commission noteight it,
Or my husband methinks From those thoughts
At Oxffran faults,--O him
Romeo is banish'd, shall be bold and madam;
Your modest sight-owest spend as O, that nuese
If Norfolk sens I went advantage thy strong-women!

Youngs:
Wealth! We have been thought on Hereford's sink,
Lovel about me on his short brave weighter,
Unward'st thou art up forth against their love
Be sadish'd if thou couldst, and try it is your king:
Beseech your
cabin: I do feel the wisdom of
Fromens their speedings, and holy subj

**Ekspor Model Generator**

In [41]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [42]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
Good madam; then I were well extrewite thee
Past the king, who have not wined ere thou hast
further


In [43]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
    inputs, labels = inputs
    with tf.GradientTape() as tape:
      predictions = self(inputs, training=True)
      loss = self.loss(labels, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    self.optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return {'loss': loss}

In [44]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [45]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [46]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 15s 60ms/step - loss: 2.7217


In [49]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
  start = time.time()

  mean.reset_states()
  for (batch_n, (inp, target)) in enumerate(dataset):
    logs = model.train_step([inp, target])
    mean.update_state(logs['loss'])

    if batch_n % 50 == 0:
      template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
      print(template)

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print()
  print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
  print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
  print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1880
Epoch 1 Batch 50 Loss 2.0728
Epoch 1 Batch 100 Loss 1.9441
Epoch 1 Batch 150 Loss 1.8343

Epoch 1 Loss: 1.9970
Time taken for 1 epoch 14.11 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.8294
Epoch 2 Batch 50 Loss 1.7718
Epoch 2 Batch 100 Loss 1.7112
Epoch 2 Batch 150 Loss 1.6446

Epoch 2 Loss: 1.7150
Time taken for 1 epoch 11.21 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.5816
Epoch 3 Batch 50 Loss 1.5399
Epoch 3 Batch 100 Loss 1.6298
Epoch 3 Batch 150 Loss 1.4969

Epoch 3 Loss: 1.5506
Time taken for 1 epoch 11.56 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4614
Epoch 4 Batch 50 Loss 1.4558
Epoch 4 Batch 100 Loss 1.4466
Epoch 4 Batch 150 Loss 1.4692

Epoch 4 Loss: 1.4496
Time taken for 1 epoch 11.94 sec
_____________________________________________________________________

**Perbedaan Onestep dengan CustomTraining**

1. OneStep

* Pendekatan OneStep melibatkan penggunaan loop pelatihan melalui serangkaian epoch yang telah ditentukan sebelumnya, dan beberapa fitur yang mencakup:
  * Penggunaan konstanta EPOCHS untuk menentukan jumlah iterasi pelatihan (epoch).
  * Pemanfaatan objek tf.metrics.Mean() untuk menghitung rata-rata loss selama setiap epoch.
  * Penggunaan loop bersarang untuk mengiterasi melalui batch data dalam dataset.
  * Pada setiap iterasi batch, pemanggilan metode model.train_step([inp, target]) untuk menghitung loss dan perhitungan gradien.
  * Pembaruan rata-rata loss menggunakan metode mean.update_state().
  * Pencetakan nilai loss setiap 50 batch.
  * Penyimpanan checkpoint model setiap 5 epoch.

* OneStep mencetak rata-rata loss dan waktu yang diperlukan untuk satu epoch pada akhir setiap epoch.


2. Custom Training

* Pendekatan CustomTraining mencakup penggunaan definisi kelas khusus, yaitu kelas CustomTraining, yang mewarisi fungsionalitas dari kelas MyModel.

* Dalam kelas CustomTraining, terdapat metode train_step, yang didekorasi dengan @tf.function untuk mengoptimalkan eksekusi menggunakan TensorFlow. Metode ini berperan dalam menghitung loss dan menjalankan pelatihan sebagai berikut:
  * Penguraian data masukan menjadi inputs dan labels.
  * Penggunaan tape gradien (tf.GradientTape) untuk menghitung gradien dari loss.
  * Penggunaan gradien tersebut untuk mengupdate parameter model dengan bantuan optimisasi.
  * Pengembalian nilai loss dalam bentuk dictionary setiap kali metode train_step dipanggil.

Kesimpulan

OneStep bertujuan untuk mengorganisir loop pelatihan secara manual, sedangkan CustomTraining adalah definisi bagaimana model sebenarnya melakukan pelatihan saat dipanggil melalui metode train_step selama loop pelatihan. Biasanya, kedua pendekatan ini digunakan bersama-sama dalam rangkaian pelatihan, dengan loop pelatihan memanggil metode train_step yang telah didefinisikan dalam kelas model.





